# Import Starting Data

In [1]:
import pandas as pd #1
import os           #3

In [2]:
given_data_folder = 'data'

file_names = os.listdir(given_data_folder)
file_names.sort()
file_names

['MRegularSeasonDetailedResults.csv', 'bracket-2022.csv']

In [3]:
file_name = 'MRegularSeasonDetailedResults.csv'

In [4]:
file_path = given_data_folder + '/' + file_name 
given_df  = pd.read_csv(file_path)
print(len(given_df))

given_df.head(5)

100423


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


# Testing With The Network

In [5]:
import sys
sys.path.append("..")

In [6]:
import random
import time
import threading
import math

from copy import deepcopy

In [7]:
from network_classes.neuron      import Neuron
from network_classes.agent       import Agent
from network_classes.population  import Population

from network_classes.data_prep     import *
from network_classes.testingreport import TestingReport


from extra.progress_package import *

# Get each teams stats from a given season

### 2021 season

In [8]:
game_solutions_2021, team_dictionary_2021, headers_2021 = stats_for_season(given_df, 2021, print_report=False)

### 2022 season

In [9]:
game_solutions_2022, team_dictionary_2022, headers_2022 = stats_for_season(given_df, 2022, print_report=True)

data size: 
-----------
number of games for 2022 season:  3736 
number of columns in base data:   34 
number of rows:                   3736 


column titles and indicis: 
---------------------------
  0: Season      1: DayNum      2: WTeamID     3: WScore      4: LTeamID     5: LScore      6: WLoc      
  7: NumOT       8: WFGM        9: WFGA       10: WFGM3      11: WFGA3      12: WFTM       13: WFTA      
 14: WOR        15: WDR        16: WAst       17: WTO        18: WStl       19: WBlk       20: WPF       
 21: LFGM       22: LFGA       23: LFGM3      24: LFGA3      25: LFTM       26: LFTA       27: LOR       
 28: LDR        29: LAst       30: LTO        31: LStl       32: LBlk       33: LPF       


base stats report: 
-------------------
game_solutions length:    3736 games
team_dictionary length:    358 teams

team_dictionary[team_id] = [# of games played, [stats FOR totals], [stats AGAINST totals]]


example row:    
------------- 

  number of games played by team: 23 

   

## base stats with FOR and AGAINST

### 2021

In [10]:
# get team dictionary using custom stats
# ---------------------------------------
FA_team_dictionary_2021, FA_headers_list_2021 = for_and_against(team_dictionary_2021)


# change team dictionary to averages instead of totals
# -----------------------------------------------------
FA_team_averages_2021 = team_totals_to_averages(FA_team_dictionary_2021, FA_headers_list_2021, print_report=False)


# prepare the final input and solution data
# ------------------------------------------
FA_inputs_2021, FA_solutions_2021 = prep_inputs_and_solutions(FA_team_averages_2021, game_solutions_2021)


# set up final inputs and solutions
# ----------------------------------
inputs_2021    = FA_inputs_2021
solutions_2021 = FA_solutions_2021

### 2022

In [11]:
# get team dictionary using custom stats
# ---------------------------------------
FA_team_dictionary_2022, FA_headers_list_2022 = for_and_against(team_dictionary_2022)


# change team dictionary to averages instead of totals
# -----------------------------------------------------
FA_team_averages_2022 = team_totals_to_averages(FA_team_dictionary_2022, FA_headers_list_2022)


# prepare the final input and solution data
# ------------------------------------------
FA_inputs_2022, FA_solutions_2022 = prep_inputs_and_solutions(FA_team_averages_2022, game_solutions_2022)


# set up final inputs and solutions
# ----------------------------------
inputs_2022    = FA_inputs_2022
solutions_2022 = FA_solutions_2022

# First Test (uses threading)

### variables for stat tracking

### initialize population

In [12]:
TEST_population = Population( len(inputs_2022[0]), 10)

#for i in range(10):
    #for agent in TEST_population.agents:
        #agent.mutate(delete_neurons=True)


In [13]:
report_data = [inputs_2021, solutions_2021,
               inputs_2022, solutions_2022]

current_test = TestingReport(TEST_population)

### Training Loop

In [28]:
testing_time = 130 # (in seocnds)

current_test.run_test(testing_time, report_data)
current_test.print_report()

[################################################# ] - 100.0%                                       

            current    prev       change  
            --------   --------   --------
timer        260.83s    130.75s    130.08s
steps            13          6          7 
steps/sec      0.05       0.05       0.05 
secs/step     18.58s     21.79s     18.58s
evolu time     0.23s      0.09s      0.14s
score time   260.60s    130.66s    129.94s
            --------   --------   --------
conf score     1251       1202         49 
base score     2403       2379         24 
accuracy      64.32%     63.68%      0.64%

		
id - [ c r, nc r, nc w,  c w] | ovrl |  conf  (1251)
---------------------------------------------
 0 - [2298,   56,   39, 1343] | 2354 |  1113
 1 - [2118,  285,  143, 1190] | 2403 |  1251
 2 - [2360,   19,   20, 1337] | 2379 |  1202
 3 - [2254,   14,   11, 1457] | 2268 |   791
 4 - [2040,  322,  247, 1127] | 2362 |  1172
 5 - [2157,  149,  153, 1277] | 2306 |   987
 6 - [235

In [29]:
current_test.print_report()



            current    prev       change  
            --------   --------   --------
timer        260.83s    130.75s    130.08s
steps            13          6          7 
steps/sec      0.05       0.05       0.05 
secs/step     18.58s     21.79s     18.58s
evolu time     0.23s      0.09s      0.14s
score time   260.60s    130.66s    129.94s
            --------   --------   --------
conf score     1251       1202         49 
base score     2403       2379         24 
accuracy      64.32%     63.68%      0.64%

		
id - [ c r, nc r, nc w,  c w] | ovrl |  conf  (1251)
---------------------------------------------
 0 - [2298,   56,   39, 1343] | 2354 |  1113
 1 - [2118,  285,  143, 1190] | 2403 |  1251
 2 - [2360,   19,   20, 1337] | 2379 |  1202
 3 - [2254,   14,   11, 1457] | 2268 |   791
 4 - [2040,  322,  247, 1127] | 2362 |  1172
 5 - [2157,  149,  153, 1277] | 2306 |   987
 6 - [2354,   18,   10, 1354] | 2372 |  1168
 7 - [2301,   59,   51, 1325] | 2360 |  1138
 8 - [2222,   21,  

In [30]:
current_test.scoring_time_report()


            time     % total
            -------  -------
total        260.6s
calc val      56.7s   21.76%
send time   172.96s   66.37%
set time      6.26s     2.4%
inv tanh      1.39s    0.53%
            -------  -------
acc for     237.31s   91.06%
            -------  -------
thread time  260.6s   100.0%
creating         0s     0.0%
starting         0s     0.0%
joining          0s     0.0%



In [ ]:
'''
            time     % total
            -------  -------
total       130.66s
calc val     28.53s   21.84%
send time    86.41s   66.13%
set time      3.25s    2.49%
inv tanh      0.75s    0.57%
            -------  -------
acc for     118.94s   91.03%
            -------  -------
thread time 130.66s   100.0%
creating         0s     0.0%
starting         0s     0.0%
joining          0s     0.0%



            time     % total
            -------  -------
total        260.6s
calc val      56.7s   21.76%
send time   172.96s   66.37%
set time      6.26s     2.4%
inv tanh      1.39s    0.53%
            -------  -------
acc for     237.31s   91.06%
            -------  -------
thread time  260.6s   100.0%
creating         0s     0.0%
starting         0s     0.0%
joining          0s     0.0%
''';

In [33]:
'''
            time     % total
            -------  -------
total       151.27s
calc val     34.37s   22.72%
send time    97.26s    64.3%
set time      4.18s    2.76%
inv tanh      1.08s    0.71%
            -------  -------
acc for     136.89s   90.49%
            -------  -------
thread time 151.27s   100.0%
creating         0s     0.0%
starting         0s     0.0%
joining          0s     0.0%



            time     % total
            -------  -------
total       305.62s
calc val      69.5s   22.74%
send time   196.67s   64.35%
set time      8.44s    2.76%
inv tanh      2.26s    0.74%
            -------  -------
acc for     276.87s   90.59%
            -------  -------
thread time 305.62s   100.0%
creating         0s     0.0%
starting         0s     0.0%
joining          0s     0.0%





            current    prev       change  
            --------   --------   --------
timer        305.93s    151.39s    154.53s
steps            12          6          6 
steps/sec      0.04       0.04       0.04 
secs/step     25.76s     25.23s     25.76s
evolu time     0.30s      0.12s      0.19s
score time   305.62s    151.27s    154.34s
            --------   --------   --------
conf score     1152        713        439 
base score     2356       2232        124 
accuracy      63.06%     59.74%      3.32%






''';

In [18]:
'''
0.37
0.36
0.36

0.69
0.73



            time     % total
            -------  -------
total        36.22s
calc val      4.48s   12.37%
send time    12.14s   33.52%
set time      0.54s    1.49%
            -------  -------
acc for      17.16s   47.38%
            -------  -------
thread time  36.22s   100.0%
creating       0.0s     0.0%
starting       0.6s    1.66%
joining      35.62s   98.34%



            time     % total
            -------  -------
total         73.8s
calc val      4.51s    6.11%
send time    12.51s   16.95%
set time      0.52s     0.7%
            -------  -------
acc for      17.54s   23.77%
            -------  -------
thread time   73.8s   100.0%
creating       0.0s     0.0%
starting      1.22s    1.65%
joining      72.58s   98.35%

''';

In [19]:
'''
            time     % total
            -------  -------
total        21.56s
calc val      7.13s   33.07%
send time    10.29s   47.73%
set time      0.55s    2.55%
            -------  -------
acc for      17.97s   83.35%
            -------  -------
thread time  21.56s   100.0%
creating       0.0s     0.0%
starting      0.21s    0.97%
joining      21.35s   99.03




            time     % total
            -------  -------
total        58.52s
calc val      5.37s    9.18%
send time     7.81s   13.35%
set time      0.43s    0.73%
            -------  -------
acc for      13.61s   23.26%
            -------  -------
thread time  58.52s   100.0%
creating       0.0s     0.0%
starting      0.87s    1.49%
joining      57.65s   98.51%





''';

In [20]:
TEST_population.num_calculations

60

In [31]:
current_test.print_top_agent()

format:

 # neurons    |  avg conn weight     
 ------------ |  ---------------     
 average from |  (# to, # from)  


  26 |   1.05   0.62   0.85   0.51   0.54   0.67   0.62   0.50   0.68   0.51   0.30   0.50   0.53   0.79 
 4.0 |  (1,3)  (1,4)  (1,5)  (1,4)  (1,4)  (1,4)  (1,3)  (1,4)  (1,3)  (1,4)  (1,5)  (1,5)  (1,4)  (1,4) 
      
  16 |   0.52   0.33   0.59   0.53   0.32   0.47   0.71   0.43   0.67   0.11   0.63   0.48   0.71   0.33 
 3.9 |  (4,5)  (7,3) (10,5)  (8,4)  (2,3)  (3,4)  (4,4)  (6,4)  (8,4)  (7,3)  (5,5) (11,5)  (7,3)  (6,3) 
      
  17 |   0.47   0.68   0.66   0.49   0.44   0.32   0.75   0.52   0.69   0.99   0.82   0.71   0.51   0.32 
 4.0 |  (3,5)  (4,5)  (1,3)  (7,4)  (1,4)  (3,3)  (4,4)  (4,3)  (1,3)  (3,4)  (4,4)  (6,4)  (5,5)  (3,4) 
      
  18 |   1.49   1.34   1.00   0.67   1.00   1.19   1.00   0.55   0.77   0.64   1.46   1.29   1.30   0.77 
 1.0 |  (9,1)  (6,1)  (2,1)  (6,1)  (3,1)  (3,1)  (2,1)  (1,1)  (2,1)  (9,1)  (4,1)  (4,1)  (2,1)  (2,1) 
      
   

In [32]:
current_test.size_report(all_attributes=False)

13 steps

                        kB          neurons     avg size
                        --------    --------    --------
layer[0]                   17.05          26        0.66 
layer[1]                   10.35          16        0.65 
layer[2]                    11.2          17        0.66 
layer[3]                    6.41          18        0.36 
layer[4]                    0.22           1        0.22 


                        kB          mB
                        --------    --------
total_size                 45.56        0.04



In [23]:
'''
3 steps

                        kB          neurons     avg size
                        --------    --------    --------
layer[0]                   20.21          26        0.78 
layer[1]                   12.11          18        0.67 
layer[2]                    9.33          12        0.78 
layer[3]                     6.2          16        0.39 
layer[4]                    0.26           1        0.26 

                        kB          mB
                        --------    --------
total_size                 48.43        0.05







''';

In [36]:
test = Neuron(-0.75)
test.calculate_value()
test.value

-0.6351489523872873

In [37]:
test.receiving += 1.5

In [ ]:
test.calculate_value()

In [24]:
#3220*7472
# 10*161*7472 = 12_029_920
# 10*322*7472 = 24_059_840

In [25]:
'''
deletion_time = 0
for i in range(len(TEST_population.agents)):
    deletion_time += TEST_population.agents[i].time_deleting
    
deletion_time
''';

In [26]:
'''
randint_time = 0
for i in range(10000):
    start_time = time.time()
    random.randint(0, 1000)
    randint_time += time.time() - start_time

random_time = 0
for i in range(10000):
    start_time = time.time()
    math.floor(1000*random.random())
    random_time += time.time() - start_time
    
uniform_time = 0
for i in range(10000):
    start_time = time.time()
    random.uniform(0,1)
    uniform_time += time.time() - start_time
    
    
print( "random.randint(a,b) : {}s".format(randint_time ) )
print( "random.random()     : {}s".format(random_time  ) )
print( "random.uniform()    : {}s".format(uniform_time ) )
''';

## Testing with splitting the input data into 10 parts

In [27]:
''' 
split_count = 10

inputs_splits = []
solutions_splits = []

for i in range(split_count):
    actual_length = len(inputs) // 2
    split_length  = (actual_length // split_count) * 2
    
    start_index =  i    * split_length
    end_index   = (i+1) * split_length
    
    inputs_splits.append(    inputs   [start_index:end_index].copy() )
    solutions_splits.append( solutions[start_index:end_index].copy() )
    
    print(len( inputs[start_index:end_index] ))
    
TEST_inputs = inputs[:2615].copy()
TEST_solutions = solutions[:2615].copy()
''';